Task 1 - Install Spark, load required libraries, set environment variables, initiate Spark, load file

In [24]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark

In [48]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
import findspark
findspark.init()
from google.colab import files
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import isnan, when, count, col, lit
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder


In [49]:
sc = SparkSession.builder.master("local[*]").getOrCreate()

In [27]:
files.upload()

Saving data.csv to data (1).csv


{'data.csv': b'Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP\nBMW,1 Series M,2011,premium unleaded (required),335,6,MANUAL,rear wheel drive,2,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135\nBMW,1 Series,2011,premium unleaded (required),300,6,MANUAL,rear wheel drive,2,"Luxury,Performance",Compact,Convertible,28,19,3916,40650\nBMW,1 Series,2011,premium unleaded (required),300,6,MANUAL,rear wheel drive,2,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350\nBMW,1 Series,2011,premium unleaded (required),230,6,MANUAL,rear wheel drive,2,"Luxury,Performance",Compact,Coupe,28,18,3916,29450\nBMW,1 Series,2011,premium unleaded (required),230,6,MANUAL,rear wheel drive,2,Luxury,Compact,Convertible,28,18,3916,34500\nBMW,1 Series,2012,premium unleaded (required),230,6,MANUAL,rear wheel drive,2,"Luxury,Performance",Compact,Coupe,28,1

In [54]:
!ls

data.csv  sample_data		     spark-2.4.8-bin-hadoop2.7.tgz
pipeline  spark-2.4.8-bin-hadoop2.7  spark-2.4.8-bin-hadoop2.7.tgz.1


In [55]:
data = sc.read.csv('data.csv',inferSchema = True, header=True)

In [56]:
data.printSchema()

root
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Engine Fuel Type: string (nullable = true)
 |-- Engine HP: integer (nullable = true)
 |-- Engine Cylinders: integer (nullable = true)
 |-- Transmission Type: string (nullable = true)
 |-- Driven_Wheels: string (nullable = true)
 |-- Number of Doors: integer (nullable = true)
 |-- Market Category: string (nullable = true)
 |-- Vehicle Size: string (nullable = true)
 |-- Vehicle Style: string (nullable = true)
 |-- highway MPG: integer (nullable = true)
 |-- city mpg: integer (nullable = true)
 |-- Popularity: integer (nullable = true)
 |-- MSRP: integer (nullable = true)



In [57]:
data.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Make,11914,None,None,Acura,Volvo
Model,11914,745.5822222222222,1490.8280590623795,1 Series,xD
Year,11914,2010.384337753903,7.5797398875957995,1990,2017
Engine Fuel Type,11911,None,None,diesel,regular unleaded
Engine HP,11845,249.38607007176023,109.19187025917194,55,1001
Engine Cylinders,11884,5.628828677213059,1.78055934824622,0,16
Transmission Type,11914,None,None,AUTOMATED_MANUAL,UNKNOWN
Driven_Wheels,11914,None,None,all wheel drive,rear wheel drive
Number of Doors,11908,3.4360933825999327,0.8813153865835529,2,4


In [58]:
# To clean the first non values
def replace(column ,value):
  return when(column != value, column).otherwise(lit(None))


In [59]:
data = data.withColumn("Market Category", replace(col("Market Category"), "N/A"))

In [60]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+
|Make|Model|Year|Engine Fuel Type|Engine HP|Engine Cylinders|Transmission Type|Driven_Wheels|Number of Doors|Market Category|Vehicle Size|Vehicle Style|highway MPG|city mpg|Popularity|MSRP|
+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+
|   0|    0|   0|               3|       69|              30|                0|            0|              6|           3742|           0|            0|          0|       0|         0|   0|
+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+



In [61]:
data = data.drop('Market Category')
data = data.na.drop()

In [62]:
print((data.count(),len(data.columns)))

(11812, 15)


In [63]:
assembler = VectorAssembler(inputCols = ['Year', 'Engine HP', 'Engine Cylinders', 
                                         'Number of Doors', 'highway MPG', 'city mpg', 'Popularity' ],
                            outputCol = 'Attributes'
                            
                            )

In [64]:
regressor = RandomForestRegressor(featuresCol='Attributes', labelCol='MSRP')

The regressor is Random Forest

In [65]:
pipeline = Pipeline(stages=[assembler, regressor])


In [66]:
pipeline.write().overwrite().save("pipeline")

In [67]:
!ls

data.csv  sample_data		     spark-2.4.8-bin-hadoop2.7.tgz
pipeline  spark-2.4.8-bin-hadoop2.7  spark-2.4.8-bin-hadoop2.7.tgz.1


In [68]:
pipelineModel = Pipeline.load('pipeline')

In [69]:
paramGrid = ParamGridBuilder()\
  .addGrid(regressor.numTrees,[100,500])\
  .build()

In [70]:
crossval = CrossValidator(estimator=pipelineModel,
                          estimatorParamMaps= paramGrid,
                          evaluator = RegressionEvaluator(labelCol='MSRP'),
                          numFolds=3)

In [71]:
train_data, test_data = data.randomSplit([0.8,0.2], seed = 123)

In [72]:
cvModel = crossval.fit(train_data)

In [73]:
bestModel = cvModel.bestModel

In [75]:
for x in range(len(bestModel.stages)):
  print(bestModel.stages[x])

VectorAssembler_40913033b866
RandomForestRegressionModel (uid=RandomForestRegressor_150431351490) with 500 trees


In [79]:
pred = cvModel.transform(test_data)

In [80]:
pred.select('MSRP', 'prediction').show()

+-----+------------------+
| MSRP|        prediction|
+-----+------------------+
|29980|32659.265580987492|
|28030|32612.481381728016|
|30030|32612.481381728016|
|32700| 37147.58884573858|
|29350| 23636.17303040264|
|31890| 27741.53532106452|
|34980| 27741.53532106452|
| 2799| 5020.895179938968|
| 2827| 5330.720310702733|
| 3381| 6133.946294174011|
|24450|26944.488743638656|
|21050|23445.063663836987|
| 2000|  5935.04173900177|
| 2181|  8443.61733433937|
| 2144|5895.1828870111185|
| 2265| 8459.214870443913|
|56780|39713.158667982505|
|49440|39682.795250606665|
|50640|39682.795250606665|
|52640|39682.795250606665|
+-----+------------------+
only showing top 20 rows



In [81]:
eval = RegressionEvaluator (labelCol = 'MSRP')

In [90]:
rmse = eval.evaluate(pred)
mse = eval.evaluate(pred ,{eval.metricName: "mse"})
mae = eval.evaluate(pred,{eval.metricName:"mae"})
r2 =  eval.evaluate(pred,{eval.metricName:"r2"})

In [91]:


print("RMSE: %.3f" %rmse)
print("MSE:   %3.f" %mse)
print("MAE:   %3.f" %mae)
print("r2:    %3.f" %r2)

RMSE: 16606.092
MSE:   275762280
MAE:   8328
r2:      1
